In [1]:
import tensorflow as tf

elements = [
    [0],
    [1, 2, 3, 4],
    [5, 6, 7],
    [7, 8, 9, 10, 11],
    [13, 14, 15, 16, 19, 20],
    [21, 22],
]
dataset = tf.data.Dataset.from_generator(
    lambda: elements, tf.int64, output_shapes=[None]
)
dataset = dataset.bucket_by_sequence_length(
    element_length_func=lambda elem: tf.shape(elem)[0],
    bucket_boundaries=[3, 5],
    bucket_batch_sizes=[2, 2, 2],
)
for elem in dataset.as_numpy_iterator():
    print(elem)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

[[1 2 3 4]
 [5 6 7 0]]
[[ 7  8  9 10 11  0]
 [13 14 15 16 19 20]]
[[ 0  0]
 [21 22]]


2023-01-20 10:25:00.253254: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-20 10:25:00.253396: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-01-20 10:25:00.299219: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [8]:
cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.resnet.ResNet101(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,
)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64)


Epoch 1/5


2023-01-20 11:13:26.694400: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 136s 155ms/step - loss: 4.7972 - accuracy: 0.0502
Epoch 2/5
782/782 [==============================] - 111s 141ms/step - loss: 4.2473 - accuracy: 0.1200
Epoch 3/5
782/782 [==============================] - 111s 142ms/step - loss: 3.7662 - accuracy: 0.1784
Epoch 4/5
782/782 [==============================] - 111s 142ms/step - loss: 4.2875 - accuracy: 0.1257
Epoch 5/5
782/782 [==============================] - 111s 142ms/step - loss: 3.6596 - accuracy: 0.1803


In [2]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices("GPU")))


Num GPUs Available:  1


In [3]:
%%time
import tensorflow as tf
import tensorflow_datasets as tfds
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label
batch_size = 128
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)
model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)

/opt/homebrew/Caskroom/miniforge/base/envs/train/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TensorFlow version: 2.10.0
Num GPUs Available:  1
Cause: Unable to locate the source code of <function normalize_img at 0x2bce4c4c0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function normalize_img at 0x2bce4c4c0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function normalize_img at 0x2bce4c4c0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/12


2023-01-20 10:25:14.921619: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - ETA: 0s - loss: 0.1504 - accuracy: 0.9554

2023-01-20 10:25:23.932715: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 10s 18ms/step - loss: 0.1504 - accuracy: 0.9554 - val_loss: 0.0617 - val_accuracy: 0.9806
Epoch 2/12
469/469 [==============================] - 8s 17ms/step - loss: 0.0423 - accuracy: 0.9867 - val_loss: 0.0389 - val_accuracy: 0.9869
Epoch 3/12
469/469 [==============================] - 8s 17ms/step - loss: 0.0263 - accuracy: 0.9921 - val_loss: 0.0390 - val_accuracy: 0.9874
Epoch 4/12
469/469 [==============================] - 8s 17ms/step - loss: 0.0184 - accuracy: 0.9944 - val_loss: 0.0459 - val_accuracy: 0.9848
Epoch 5/12
469/469 [==============================] - 8s 17ms/step - loss: 0.0133 - accuracy: 0.9956 - val_loss: 0.0583 - val_accuracy: 0.9833
Epoch 6/12
469/469 [==============================] - 8s 17ms/step - loss: 0.0091 - accuracy: 0.9970 - val_loss: 0.0436 - val_accuracy: 0.9881
Epoch 7/12
469/469 [==============================] - 8s 17ms/step - loss: 0.0060 - accuracy: 0.9979 - val_loss: 0.0491 - val_accuracy: 0.9869
Epoch 8/1